###Project Summary
<img src ='https://images.unsplash.com/photo-1548859047-1d15def63a14?ixlib=rb-1.2.1&ixid=MXwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHw%3D&auto=format&fit=crop&w=2700&q=80' width="800" height = "200">
######Ivy(Ruxin) Tong | November 15th, 2020 | Image courtesy of Ivan Olenkevich

####**Objective** : Address the Challenge of Evaluation and Prediction of American’s Mobility under Extreme Weather Events






####**Datasets** :  Weather_CBG_2019 | Social Distancing Metrics v2.1 
######***About the data***
- Dataset - Weather_CBG_2019 records daily weather parameters by census block group in the United States. There're 7 variables. 

| Variable   | Description  | Unit | 
|-----------------  |---------------|------|
| geoid    | census block group ID |  | 
| precip  | daily precipitation | mm|
| rmax | maximum daily relative humidity | % |
| rmin | mminimum daily relative humidity | % |
| srad | surface downwelling solar radiation | W/m^2 |
| tmax | maximum daily temperature | degress F |
| tmin | minimum daily temperature | degress F |
| wind_speed | wind speed | mph|


- Dataset - Social Distancing Metrics v2.1 is a product of Safegraph which aggregately summarizes daily views of USA foot-traffic between census block groups. There're 23 variables. For this analysis, I mainly use
  
  
| Variable   | Description  | Unit | 
|-----------------  |---------------|------|
| origin_census_block_group    | 12-digit FIPS code for the Census Block Group |  | 
| distance_traveled_from_home  | Median distance traveled from the geohash-7 of the home by the devices measured within the time period. All distance with 0 has been excluded. | m |
| distance_traveled_from_home  | Median distance traveled from the geohash-7 of the home by the devices measured within the time period. All distance with 0 has been excluded. | m |


  
  
######***Acknowledgements*** : Social Distancing Metrics v2.1 is downloaded from Safegraph



####**Conclusion**:




####**Model and Result**:

#### Analysis
###### Step 0 : Install and Import Relevant Packages

In [4]:
%run ./Packages_setup


###### Step 1 : Data Preparation and Analysis for weather_cbg_2019
  - connect AWS S3 Bucket to Databricks and read the dataset

In [6]:
# # Secure access to S3 buckets using instance profiles
# awsAccessKey = ""
# # Encode the Secret Key to remove any "/" characters
# secretKey = "".replace("/", "%2F")
# awsBucketName = "weathercbg2019"
# mountPoint = f"/mnt/weathercbg"


# mountTarget = "s3a://{}:{}@{}".format(awsAccessKey, secretKey, awsBucketName)
# dbutils.fs.mount(mountTarget, mountPoint)




In [7]:
# dbutils.fs.ls("/mnt/weathercbg")

In [8]:
# dbutils.fs.head("dbfs:/mnt/weathercbg/weather_cbg_2019.csv",1000)

In [9]:
# # #read weathercbgDF

# weathercbgDF = (spark.read
#   .option("delimiter", ",")
#   .option("header", True)
#   .option("inferSchema", True)
#   .csv("/mnt/weathercbg/weather_cbg_2019.csv")
# )


- Transform and Explore the weather dataset

In [11]:
# #  Add three date-related variables
# weathercbgDF = weathercbgDF.withColumn("month", F.month('date'))
# weathercbgDF = weathercbgDF.withColumn("day", F.dayofmonth('date'))
# weathercbgDF = weathercbgDF.withColumn("year", F.year('date'))
# weathercbgDF.show(5)
# # save to local folder to easy access later and cheaper. 
# weathercbgDF.write.format('com.databricks.spark.csv').save("/FileStore/weathercbgDF.csv",header = 'true',inferSchema = "true")

In [12]:
# #   unmount s3
# try:
#   dbutils.fs.unmount(mountPoint)
# except:
#   print("{} already unmounted".format(mountPoint))

In [13]:
def spark_df_shape(self):
    return (self.count(),len(self.columns))
  
pyspark.sql.dataframe.DataFrame.shape = spark_df_shape


weathercbgDF = spark.read.format("csv").option("header", "true").option("inferSchema","true").load("/FileStore/weathercbgDF.csv")
print("weather cbg 2019 dataset")
weathercbgDF.printSchema()
print("shape:")
print(weathercbgDF.shape())


In [14]:
# from pyspark.sql.functions import min, max

# display(
#   weathercbgDF.select(min("date"), max("date"))
# )

- Create summary table group by month and geoid & Basic Visualization

In [16]:
# #  create monthly summary table group by geoid, for visualization and faster operation

# weather_cbg_month = (weathercbgDF.groupBy("geoid","month")
#                             .agg(
#                               mean("precip").alias("mean_precip"),
#                               mean("rmax").alias("mean_rmax"),
#                               mean("rmin").alias("mean_rmin"),
#                               mean("srad").alias("mean_srad"),
#                               mean("tmin").alias("mean_tmin"),
#                               mean("tmax").alias("mean_tmax"),
#                               mean("wind_speed").alias("mean_wind_speed"))
#                              .sort(["geoid","month"]))

# weather_cbg_month.show(5)


# weather_cbg_month.write.format('com.databricks.spark.csv').save("/FileStore/weather_cbg_month.csv",header = 'true',inferSchema = 'true')

In [17]:
# Generate a sample for exploratory data analysis
# weather_cbg_month = spark.read.format("csv").option("header", "true",).option("inferSchema", "true").load("/FileStore/weather_cbg_month.csv")
# weather_cbg_month.printSchema()

# weather_cbg_2019_month_sample =weather_cbg_month.sample(0.01).toPandas()
# weather_cbg_2019_month_sample.head()


In [18]:
# Understand distribution of Variables
# variables = weather_cbg_2019_month_sample.drop(["geoid","month"], axis = 1)
# fig,ax = plt.subplots(1,7, figsize= (30,5))

# for i, variable in enumerate(variables):
#   sns.distplot(variables[variable], ax=ax[i])

In [19]:
# these distributions are not on the same scale, standardize them
# fix, ax = plt.subplots(1,7,figsize = (25,5))

# for i, variable in enumerate(variables):
#   original_variables = variables[variable]
#   variable_scaled = (original_variables - original_variables.mean())/original_variables.std()
#   sns.distplot(variable_scaled, ax=ax[i])
#   ax[i].set_xlim(-2,2)

In [20]:
#correlation
# sns.pairplot(variables)


In [21]:
# corr = variables.corr()
# mask = np.zeros_like(corr)
# mask[np.triu_indices_from(mask,0)] = True
# sns.heatmap(corr, mask = mask,square = True, annot = True)

In [22]:
# # scaled data by month. This is generally the US case, it varies by the region
# for i, variable in enumerate(variables):
#   original_variables = variables[variable]
#   variable_scaled = (original_variables - original_variables.mean())/original_variables.std()
#   sns.lineplot(x = "month", y=variable_scaled, data = weather_cbg_2019_month_sample,legend='brief', label=variable)

# plt.ylabel("values")
# plt.legend(bbox_to_anchor=(1.02, 1),borderaxespad=0.)
# plt.title("mean weather parameters level by month")

  

- Prepare Jan file for initial modeling

In [24]:
# Prepare Jan file for initial modeling
weatherDF_Jan = weathercbgDF.where(col("month") == 1).toPandas()
weatherDF_Jan.head()

,geoid,date,precip,rmax,rmin,srad,tmin,tmax,wind_speed,month,day,year
0,10730059033,2019-01-01,0.000,97.200,65.200,111.800,48.470,61.970,2.461,1,1,2019
1,10730059033,2019-01-02,20.900,86.300,81.600,30.600,48.470,52.790,3.355,1,2,2019
2,10730059033,2019-01-03,24.900,79.400,63.100,59.200,50.630,59.450,5.816,1,3,2019
3,10730059033,2019-01-04,0.900,100.000,55.700,99.100,40.550,62.150,14.316,1,4,2019
4,10730059033,2019-01-05,0.000,87.600,39.200,126.800,36.950,60.170,7.382,1,5,2019


In [25]:
# weatherDF_Jan.shape

In [26]:
# weatherDF_Jan.info()

In [27]:
weatherDF_Jan.geoid.nunique()

In [28]:
weatherDF_Jan[weatherDF_Jan.geoid == 10730059033]

###### Step 2 : Data Preparation and Analysis for Social Distancing Metrics v2.1

- Get data from Safegraph
    - Download this dataset from safegraph through cluster terminal to filepath 'cd databricks/driver/mnt/' -> Move them to '/dbfs/mnt#'

In [31]:
# use this command to move files so that I can directly access from databricks data UI
# dbutils.fs.mv(r"file:/mnt/", r"dbfs:/social_distance_data/", True)

In [32]:
%fs ls /social_distance_data/social_distance/2019/

In [33]:
%fs ls /social_distance_data/social_distance/2019/01

In [34]:
%fs ls /social_distance_data/social_distance/2019/01/01

- Preparing SafeGraph social-distancing data from daily to monthly

In [36]:
# initially i tried to use pyspark to read, but in this case, i found panda is better for displaying columns with json data type
# start with 1/1/2019 file, just to undertand the dataset
# sd20190101 = pd.read_csv("/dbfs/social_distance_data/social_distance/2019/01/01/2019-01-01-social-distancing.csv.gz",compression = "gzip")
# sd20190101.head() 

In [37]:
# sd20190101.shape

In [38]:
# sd20190101.info()

In [39]:
# Prepare January file for initial modeling
import glob, os 
Jan_files = glob.glob("/dbfs/social_distance_data/social_distance/2019/01/*/*")

li= []
for file in Jan_files:
    df = pd.read_csv(file, compression='gzip', usecols=['origin_census_block_group','date_range_start','date_range_end','distance_traveled_from_home','mean_home_dwell_time','completely_home_device_count','device_count'])
    li.append(df)

sd_data_jan = pd.concat(li, axis=0, ignore_index=True)

sd_data_jan


,origin_census_block_group,date_range_start,date_range_end,device_count,distance_traveled_from_home,completely_home_device_count,mean_home_dwell_time
0,10330210004,2019-01-01T00:00:00-06:00,2019-01-02T00:00:00-06:00,78,687,28,721
1,10730049022,2019-01-01T00:00:00-06:00,2019-01-02T00:00:00-06:00,259,5381,93,292
2,11210118001,2019-01-01T00:00:00-06:00,2019-01-02T00:00:00-06:00,91,1591,45,588
3,11250106021,2019-01-01T00:00:00-06:00,2019-01-02T00:00:00-06:00,392,4623,171,770
4,21880002003,2019-01-01T00:00:00-09:00,2019-01-02T00:00:00-09:00,10,0,3,857
...,...,...,...,...,...,...,...
6805910,550590013003,2019-01-31T00:00:00-06:00,2019-02-01T00:00:00-06:00,52,1191,31,816
6805911,550590020001,2019-01-31T00:00:00-06:00,2019-02-01T00:00:00-06:00,65,4145,22,799
6805912,550679605002,2019-01-31T00:00:00-06:00,2019-02-01T00:00:00-06:00,43,546,19,707
6805913,550730019007,2019-01-31T00:00:00-06:00,2019-02-01T00:00:00-06:00,41,4348,16,508


In [40]:
sd_data_jan['date_range_start'] = pd.to_datetime(sd_data_jan['date_range_start'],utc= True)
sd_data_jan['date_range_end'] = pd.to_datetime(sd_data_jan['date_range_end'],utc= True)
sd_data_jan['month'] = sd_data_jan['date_range_start'].dt.month
sd_data_jan['day'] = sd_data_jan['date_range_start'].dt.day
sd_data_jan['date_time'] = sd_data_jan['date_range_end']-sd_data_jan['date_range_start']
sd_data_jan['date'] = [d.date() for d in sd_data_jan['date_range_start']]
sd_data_jan['ratio_not_leaving'] = sd_data_jan['completely_home_device_count']/sd_data_jan['device_count']
sd_data_jan = sd_data_jan.drop(['date_range_start','date_range_end'],1)

In [41]:
sd_data_jan.head()

,origin_census_block_group,device_count,distance_traveled_from_home,completely_home_device_count,mean_home_dwell_time,month,day,date_time,date,ratio_not_leaving
0,10330210004,78,687,28,721,1,1,1 days,2019-01-01,0.359
1,10730049022,259,5381,93,292,1,1,1 days,2019-01-01,0.359
2,11210118001,91,1591,45,588,1,1,1 days,2019-01-01,0.495
3,11250106021,392,4623,171,770,1,1,1 days,2019-01-01,0.436
4,21880002003,10,0,3,857,1,1,1 days,2019-01-01,0.300


In [42]:
sd_data_jan.shape

Out[33]: (6805915, 10)

In [43]:
sd_data_jan.describe()

In [44]:
sd_data_jan.info()

In [45]:
sd_data_jan[sd_data_jan.origin_census_block_group == 10730059033]

In [46]:
sd_data_jan.origin_census_block_group.nunique()  # 219759
# sd_data_jan.origin_census_block_group[~sd_data_jan.origin_census_block_group.isin(weatherDF_Jan.geoid)].nunique() # 4869

###### Step 3 : Join two Jan files together

In [48]:
merge_Jan = pd.merge(sd_data_jan, weatherDF_Jan, how='inner', left_on=['origin_census_block_group','day'], right_on = ['geoid','day'])

In [49]:
merge_Jan.head()

,origin_census_block_group,device_count,distance_traveled_from_home,completely_home_device_count,mean_home_dwell_time,month_x,day,date_time,date_x,ratio_not_leaving,geoid,date_y,precip,rmax,rmin,srad,tmin,tmax,wind_speed,month_y,year
0,10330210004,78,687,28,721,1,1,1 days,2019-01-01,0.359,10330210004,2019-01-01,0.400,99.900,67.600,110.300,43.430,56.390,6.263,1,2019
1,10730049022,259,5381,93,292,1,1,1 days,2019-01-01,0.359,10730049022,2019-01-01,0.000,94.700,66.500,111.900,50.270,62.510,2.908,1,2019
2,11210118001,91,1591,45,588,1,1,1 days,2019-01-01,0.495,11210118001,2019-01-01,0.000,98.200,64.300,99.600,49.370,63.770,2.908,1,2019
3,11250106021,392,4623,171,770,1,1,1 days,2019-01-01,0.436,11250106021,2019-01-01,0.000,94.800,65.800,117.700,50.090,63.050,4.026,1,2019
4,40019450014,97,615,33,204,1,1,1 days,2019-01-01,0.340,40019450014,2019-01-01,0.600,100.000,69.300,87.000,-3.730,25.790,3.803,1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6656666,550250019002,52,1852,29,670,1,31,1 days,2019-01-31,0.558,550250019002,2019-01-31,1.000,100.000,48.600,124.300,-30.370,-2.830,7.382,1,2019
6656667,550590013003,52,1191,31,816,1,31,1 days,2019-01-31,0.596,550590013003,2019-01-31,0.000,100.000,56.100,85.100,-25.150,2.930,8.948,1,2019
6656668,550590020001,65,4145,22,799,1,31,1 days,2019-01-31,0.338,550590020001,2019-01-31,0.000,100.000,55.100,83.400,-24.430,2.930,9.843,1,2019
6656669,550679605002,43,546,19,707,1,31,1 days,2019-01-31,0.442,550679605002,2019-01-31,0.000,100.000,52.900,118.900,-30.550,-4.630,9.843,1,2019


In [50]:
merge_Jan[merge_Jan.geoid==10730059033]

,origin_census_block_group,device_count,distance_traveled_from_home,completely_home_device_count,mean_home_dwell_time,month_x,day,date_time,date_x,ratio_not_leaving,geoid,date_y,precip,rmax,rmin,srad,tmin,tmax,wind_speed,month_y,year
119438,10730059033,159,2770,76,693,1,1,1 days,2019-01-01,0.478,10730059033,2019-01-01,0.000,97.200,65.200,111.800,48.470,61.970,2.461,1,2019
316193,10730059033,160,4737,66,689,1,2,1 days,2019-01-02,0.412,10730059033,2019-01-02,20.900,86.300,81.600,30.600,48.470,52.790,3.355,1,2019
536561,10730059033,172,2594,65,614,1,3,1 days,2019-01-03,0.378,10730059033,2019-01-03,24.900,79.400,63.100,59.200,50.630,59.450,5.816,1,2019
783345,10730059033,164,3202,50,598,1,4,1 days,2019-01-04,0.305,10730059033,2019-01-04,0.900,100.000,55.700,99.100,40.550,62.150,14.316,1,2019
978619,10730059033,153,3017,64,680,1,5,1 days,2019-01-05,0.418,10730059033,2019-01-05,0.000,87.600,39.200,126.800,36.950,60.170,7.382,1,2019
1187443,10730059033,162,2106,68,711,1,6,1 days,2019-01-06,0.420,10730059033,2019-01-06,0.000,95.800,33.200,119.400,36.770,66.830,2.908,1,2019
1385368,10730059033,157,3435,57,622,1,7,1 days,2019-01-07,0.363,10730059033,2019-01-07,0.400,83.800,38.300,126.500,44.150,67.370,7.158,1,2019
1619164,10730059033,167,2712,70,594,1,8,1 days,2019-01-08,0.419,10730059033,2019-01-08,0.000,100.000,43.900,116.400,42.710,68.990,8.948,1,2019
1810690,10730059033,165,3208,52,541,1,9,1 days,2019-01-09,0.315,10730059033,2019-01-09,0.000,68.600,34.000,135.000,31.730,51.530,10.961,1,2019
2031434,10730059033,162,2307,53,619,1,10,1 days,2019-01-10,0.327,10730059033,2019-01-10,0.000,60.100,31.100,133.500,27.410,45.590,8.053,1,2019


In [51]:
# understand predictor  = distance traveled, highly right skewed
plt.hist(sd_data_jan.distance_traveled_from_home, bins = range(0, 1000000,500), log = True)
plt.show()

In [52]:
# Try a cut off of 50000 meters
plt.hist(sd_data_jan.distance_traveled_from_home, bins = range(0, 500000,500), log = True)
plt.show()

In [53]:
merge_Jan = merge_Jan[merge_Jan.distance_traveled_from_home < 50000]

In [54]:
merge_Jan.describe()

,origin_census_block_group,device_count,distance_traveled_from_home,completely_home_device_count,mean_home_dwell_time,month_x,day,date_time,ratio_not_leaving,geoid,precip,rmax,rmin,srad,tmin,tmax,wind_speed,month_y,year
count,6646227.000,6646227.000,6646227.000,6646227.000,6646227.000,6646227.000,6646227.000,6646227,6646227.000,6646227.000,6646227.000,6646227.000,6646227.000,6646227.000,6646227.000,6646227.000,6646227.000,6646227.000,6646227.000
mean,282365155950.630,104.975,2741.600,38.811,670.706,1.000,16.003,1 days 00:00:00,0.368,282365155950.630,2.878,84.206,47.887,98.898,27.575,45.491,10.120,1.000,2019.000
std,157041644081.377,103.829,2764.692,47.043,110.674,0.000,8.943,0 days 00:00:00,0.107,157041644081.377,7.424,16.065,16.457,38.815,16.413,17.336,4.819,0.000,0.000
min,10010201001.000,5.000,0.000,1.000,0.000,1.000,1.000,1 days 00:00:00,0.003,10010201001.000,0.000,21.400,1.100,13.600,-45.850,-24.790,0.671,1.000,2019.000
25%,131198901012.000,55.000,1279.000,19.000,609.000,1.000,8.000,1 days 00:00:00,0.293,131198901012.000,0.000,74.200,36.000,69.200,18.950,33.530,6.487,1.000,2019.000
50%,281059506012.000,80.000,2095.000,29.000,676.000,1.000,16.000,1 days 00:00:00,0.367,281059506012.000,0.000,87.700,47.300,98.300,28.670,45.770,9.171,1.000,2019.000
75%,410510025014.000,122.000,3284.000,46.000,740.000,1.000,24.000,1 days 00:00:00,0.440,410510025014.000,1.400,100.000,60.100,126.200,38.750,58.910,12.974,1.000,2019.000
max,560459513003.000,37862.000,49991.000,34014.000,1393.000,1.000,31.000,1 days 00:00:00,0.949,560459513003.000,223.800,100.000,100.000,247.000,77.810,88.250,42.502,1.000,2019.000


- Feature Engineering

In [56]:
y = merge_Jan

In [59]:
# sd_data = spark.read.format("csv").option("header", "true").option("inferSchema","true").option("sep", ",").load("/social_distance_data/social_distance/2019/*/*")

In [60]:
# take too long to run
# # prepare all 2019 social distance data
# import glob, os 
# all_files = glob.glob("/dbfs/social_distance_data/social_distance/2019/*/*/*")

# li= []
# for file in all_files:
#     df = pd.read_csv(file, compression='gzip', usecols=['origin_census_block_group','date_range_start','date_range_end','distance_traveled_from_home','mean_home_dwell_time','completely_home_device_count','device_count'])
#     df['date_range_start'] = pd.to_datetime(df['date_range_start'],utc= True)
#     df['date_range_end'] = pd.to_datetime(df['date_range_end'],utc= True)
#     df['month'] = df['date_range_start'].dt.month
#     df['day'] = df['date_range_start'].dt.day
#     df['date_time'] = df['date_range_end']-df['date_range_start']
#     df['ratio_not_leaving'] = df['completely_home_device_count']/df['device_count']
#     df = df.drop(['date_range_start','date_range_end'],1)
    
#     li.append(df)

# sd_data = pd.concat(li, axis=0, ignore_index=True).to_csv('/dbfs/tmp/sd_2019.csv',index=False,columns=cols)

# del all_files
# del li
